<a href="https://colab.research.google.com/github/sakshyamaryal/MAD-mobile-app-development-/blob/main/AI_assingment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [45]:
dataset_path = '/content/sample_data/dataset'


# Step 1: Dataset Loading
labels = os.listdir(dataset_path)
images = []
labels_encoded = []

for label in labels:
    images_path = os.path.join(dataset_path, label)
    for image_name in os.listdir(images_path):
        image_path = os.path.join(images_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (64, 64))  # Resize the image to a consistent size
        image_array = np.expand_dims(image, axis=-1)  # Add channel dimension
        images.append(image_array)
        labels_encoded.append(labels.index(label))


In [22]:
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     shear_range=0.2,
#     horizontal_flip=True,
#     channel_shift_range=50,
#     brightness_range=[0.8, 1.2],
#     zoom_range=0.2,
#     fill_mode='nearest'
# )


In [46]:
# Step 2: Data Augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     shear_range=0.2,
#     horizontal_flip=True,
#     channel_shift_range=50,
#     brightness_range=[0.8, 1.2],
#     zoom_range=0.2,
#     fill_mode='nearest'
# )
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

augmented_images = []
augmented_labels = []

# Create directories to save augmented images
augmented_path = '/content/sample_data/augmented_images'

os.makedirs(augmented_path, exist_ok=True)

for image, label in zip(images, labels_encoded):
    augmented_images.append(image)
    augmented_labels.append(label)
    image = image.reshape((1,) + image.shape)
    augmented = datagen.flow(image, batch_size=1)
    i = 0
    for batch in augmented:
        augmented_image = batch[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append(label)

        # Save augmented images
        save_path = os.path.join(augmented_path, f"{label}_{i}.jpg")
        cv2.imwrite(save_path, augmented_image)
        i += 1
        if i >= 100:
            break

print("Augmented Images Saved")


Augmented Images Saved


In [47]:
# X_train, X_test, y_train, y_test = train_test_split(augmented_images, augmented_labels, test_size=0.2, random_state=42)


In [48]:
X_train, X_test, y_train, y_test = train_test_split(augmented_images, augmented_labels, test_size=0.2, random_state=42)

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Step 4: Model Design
num_classes = len(labels)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [49]:
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
6/6 [==============================] - 1s 157ms/step - loss: 149.5518 - accuracy: 0.4720
Epoch 2/10
6/6 [==============================] - 1s 186ms/step - loss: 3.3564 - accuracy: 0.5093
Epoch 3/10
6/6 [==============================] - 1s 123ms/step - loss: 0.8853 - accuracy: 0.5280
Epoch 4/10
6/6 [==============================] - 1s 124ms/step - loss: 0.9727 - accuracy: 0.5342
Epoch 5/10
6/6 [==============================] - 1s 120ms/step - loss: 0.4078 - accuracy: 0.7950
Epoch 6/10
6/6 [==============================] - 1s 189ms/step - loss: 0.2742 - accuracy: 0.9068
Epoch 7/10
6/6 [==============================] - 1s 223ms/step - loss: 0.6095 - accuracy: 0.7019
Epoch 8/10
6/6 [==============================] - 1s 172ms/step - loss: 0.2693 - accuracy: 0.8882
Epoch 9/10
6/6 [==============================] - 1s 124ms/step - loss: 0.1759 - accuracy: 0.9379
Epoch 10/10
6/6 [==============================] - 1s 124ms/step - loss: 0.1367 - accuracy: 0.9565


In [50]:
# Step 6: Model Save and Selection
if not os.path.exists('/content/sample_data/face_recognition_model.h5'):
    os.makedirs('/content/sample_data/face_recognition_model.h5')
model.save('/content/sample_data/face_recognition_model.h5')


In [51]:
# Step 7: Accuracy Evaluation
accuracy = model.evaluate(np.array(X_test), np.array(y_test))[1]
print("Accuracy:", accuracy)


2/2 [==============================] - 0s 19ms/step - loss: 0.1940 - accuracy: 0.9024
Accuracy: 0.9024389982223511


In [52]:
# Step 8: Real Image Testing
def recognize_faces(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
